## Hyperparameter Optimization

In this section, all hyperparameter combinations are tested and the parameters that give the best results are found.

You should write the hyperparameter values you want to try in the corresponding list.

An important point here is that after selecting the feature vector, you should look at the input values to determine which model will use which input.

The folder in which the generated models will be saved is determined by 'export_dir= ' in the model.export() function.

In [ ]:
import os

import numpy as np
import pandas as pd
from tqdm import tqdm

import tensorflow as tf
assert tf.__version__.startswith('2')

from tflite_model_maker import model_spec
from tflite_model_maker import image_classifier
from tflite_model_maker.config import ExportFormat
from tflite_model_maker.config import QuantizationConfig
from tflite_model_maker.image_classifier import DataLoader

image_path = os.path.join(os.path.dirname("datasets/train3/"))

data = DataLoader.from_folder(image_path)
train_data, test_data = data.split(0.9)

# hyperparameter dictionary
hyperparameters = {
    'url': [],
    'epochs': [],
    'learning_rate': [],
    'train_whole_model': [],
    'shuffle': [],
    'use_augmentation': [],
    'quantization_config': [],
    'accuracy': [],
    'file_name': []
}

# lyst of hyperparameters
url_list = ['https://tfhub.dev/google/imagenet/mobilenet_v2_035_192/feature_vector/5',
            'https://tfhub.dev/google/imagenet/mobilenet_v2_050_224/feature_vector/5',
            'https://tfhub.dev/google/imagenet/mobilenet_v3_small_075_224/feature_vector/5']
epochs_list = [2,3,5,7]
train_whole_model_list = [True, None]
learning_rate_list = [0.009, 0.01, 0.013, 0.015, 0.02]
shuffle_list = [True, False]
use_augmentation_list = [True, False]
quantization_config_list = [QuantizationConfig.for_float16(), QuantizationConfig.for_dynamic()]

number = 1
for a in url_list:
    for b in epochs_list:
        for c in learning_rate_list:
            for d in train_whole_model_list:
                for e in shuffle_list:
                    for f in use_augmentation_list:
                        for g in quantization_config_list:
                            modelspec = image_classifier.ModelSpec(uri=a)
                            if a == 'https://tfhub.dev/google/imagenet/mobilenet_v2_035_192/feature_vector/5':
                                modelspec.input_image_shape = [192,192] # input values of the feature vector
                            else:
                                modelspec.input_image_shape = [224,224] # input values of the feature vector

                            model = image_classifier.create(train_data, model_spec=modelspec, epochs=b, learning_rate=c,
                                                            train_whole_model=d, shuffle=e, use_augmentation=f)

                            loss, accuracy = model.evaluate(test_data)
                            config = g
                            file_name = 'model' + str(number) + '.tflite'

                            model.export(export_dir='./hyp_models', tflite_filename=file_name, quantization_config=config)
                            # The folder in which the generated models will be saved is specified with 'export_dir= '

                            if number%2 == 1:
                                g = 'float_16'
                            else:
                                g = 'dynamic'

                            progress = number/960 * 100
                            number += 1

                            hyperparameters['url'].append(a)
                            hyperparameters['epochs'].append(b)
                            hyperparameters['learning_rate'].append(c)
                            hyperparameters['train_whole_model'].append(d)
                            hyperparameters['shuffle'].append(e)
                            hyperparameters['use_augmentation'].append(f)
                            hyperparameters['quantization_config'].append(g)
                            hyperparameters['accuracy'].append(accuracy)
                            hyperparameters['file_name'].append(file_name)

                            df = pd.DataFrame.from_dict(hyperparameters, orient='index')
                            df = df.transpose()
                            df.to_excel('hyp_models_result.xlsx')

                            print('progress: %' + str(progress))


df = pd.DataFrame.from_dict(hyperparameters, orient='index')
df = df.transpose()
df.to_excel('hyp_models_result.xlsx')
print(df)

/home/ubuntu/anaconda3/envs/tf/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:53: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.5.0 and strictly below 2.8.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.4.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


INFO:tensorflow:Load image with size: 28222, num_label: 2, labels: fincan, red.
INFO:tensorflow:Retraining the models...
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2 (HubKer (None, 1280)              410208    
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 2)                 2562      
Total params: 412,770
Trainable params: 398,690
Non-trainable params: 14,080
_________________________________________________________________
None
INFO:tensorflow:Use default resize_bicubic.


INFO:tensorflow:Use default resize_bicubic.


INFO:tensorflow:Use customized resize method bilinear


INFO:tensorflow:Use customized resize method bilinear


...

INFO:tensorflow:TensorFlow Lite model exported successfully: ./hyp_models/model3.tflite


%0.3125
INFO:tensorflow:Retraining the models...


INFO:tensorflow:Retraining the models...


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hub_keras_layer_v1v2_3 (HubK (None, 1280)              410208    
_________________________________________________________________
dropout_3 (Dropout)          (None, 1280)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 2)                 2562      
Total params: 412,770
Trainable params: 398,690
Non-trainable params: 14,080
_________________________________________________________________
None
Epoch 1/2
675/793 [========================>.....] - ETA: 11s - loss: 0.3176 - accuracy: 0.9632

KeyboardInterrupt: 